In [2]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# Extract the Apple store data set: 
apple_store = "Resources/applestore.csv"
apple_store_df = pd.read_csv(apple_store)
apple_store_df.head()

,Unnamed: 0,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,1,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,38,5,10,1
1,2,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,37,5,23,1
2,3,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,37,5,3,1
3,4,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping,37,5,9,1
4,5,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference,37,5,45,1


In [4]:
# extract the neccessary columns to be used: 
new_applestore_df = apple_store_df[['id', 'track_name', 'size_bytes','price','user_rating','prime_genre']].copy()
new_applestore_df.head()

,id,track_name,size_bytes,price,user_rating,prime_genre
0,281656475,PAC-MAN Premium,100788224,3.99,4.0,Games
1,281796108,Evernote - stay organized,158578688,0.00,4.0,Productivity
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,0.00,3.5,Weather
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,0.00,4.0,Shopping
4,282935706,Bible,92774400,0.00,4.5,Reference


In [5]:
# Extract the Apple store data set: 
apple_des = "Resources/applestore_description.csv"
apple_des_df = pd.read_csv(apple_des)
apple_des_df.head()

,id,track_name,size_bytes,app_desc
0,281656475,PAC-MAN Premium,100788224,"SAVE 20%, now only $3.99 for a limited time!\n..."
1,281796108,Evernote - stay organized,158578688,Let Evernote change the way you organize your ...
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,Download the most popular free weather app pow...
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,The eBay app is the best way to find anything ...
4,282935706,Bible,92774400,On more than 250 million devices around the wo...


In [17]:
# connect to local database: 
connection_apples = "postgres:Postgres@localhost:5432/Two_pandas_db"
engine = create_engine(f'postgresql://{connection_apples}')

In [7]:
# Use pandas to load apple store csv converted DataFrame into database: 
new_applestore_df.to_sql(name='apple_store', con=engine, if_exists='append', index=False)

In [8]:
# Use pandas to load applestore description csv converted DataFrame into database:
apple_des_df.to_sql(name='apple_des', con=engine, if_exists='append', index=False)

In [9]:
# call out the names of tables: 
engine.table_names()

/var/folders/yn/xk1nllmx5_j6zx9bwj6p24kw0000gn/T/ipykernel_41052/3530174579.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['apple_des', 'apple_store', 'apple_master']

In [10]:
# Confirm data has been added by querying the tables: 
pd.read_sql_query('select * from apple_store', con=engine).head()

,id,track_name,size_bytes,price,user_rating,prime_genre
0,281656475,PAC-MAN Premium,100788224,3.99,4.0,Games
1,281796108,Evernote - stay organized,158578688,0.00,4.0,Productivity
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,0.00,3.5,Weather
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,0.00,4.0,Shopping
4,282935706,Bible,92774400,0.00,4.5,Reference


In [11]:
pd.read_sql_query('select * from apple_des', con=engine).head()

,id,track_name,size_bytes,app_desc
0,281656475,PAC-MAN Premium,100788224,"SAVE 20%, now only $3.99 for a limited time!\n..."
1,281796108,Evernote - stay organized,158578688,Let Evernote change the way you organize your ...
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,Download the most popular free weather app pow...
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,The eBay app is the best way to find anything ...
4,282935706,Bible,92774400,On more than 250 million devices around the wo...


In [12]:
# merge to have 1 master df
master_apple_df = pd.merge(new_applestore_df,apple_des_df,how="left")
master_apple_df.head()

,id,track_name,size_bytes,price,user_rating,prime_genre,app_desc
0,281656475,PAC-MAN Premium,100788224,3.99,4.0,Games,"SAVE 20%, now only $3.99 for a limited time!\n..."
1,281796108,Evernote - stay organized,158578688,0.00,4.0,Productivity,Let Evernote change the way you organize your ...
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,0.00,3.5,Weather,Download the most popular free weather app pow...
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,0.00,4.0,Shopping,The eBay app is the best way to find anything ...
4,282935706,Bible,92774400,0.00,4.5,Reference,On more than 250 million devices around the wo...


In [13]:
# Use pandas to load master table Apple into SQL database:
master_apple_df.to_sql(name='apple_master', con=engine, if_exists='append', index=False)

In [14]:
# check if table available in SQL database: 
pd.read_sql_query('select * from apple_master', con=engine).head()

,id,track_name,size_bytes,price,user_rating,prime_genre,app_desc
0,281656475,PAC-MAN Premium,100788224,3.99,4.0,Games,"SAVE 20%, now only $3.99 for a limited time!\n..."
1,281796108,Evernote - stay organized,158578688,0.00,4.0,Productivity,Let Evernote change the way you organize your ...
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,0.00,3.5,Weather,Download the most popular free weather app pow...
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,0.00,4.0,Shopping,The eBay app is the best way to find anything ...
4,282935706,Bible,92774400,0.00,4.5,Reference,On more than 250 million devices around the wo...


In [22]:
# try another SQL way JOIN 2 tables
another_master = pd.read_sql("""select apple_store.id, apple_store.track_name, apple_store.size_bytes, 
                     apple_store.price, apple_store.user_rating, apple_store.prime_genre, 
                     apple_des.app_desc
                     from apple_store
                     join apple_des
                     on apple_store.id = apple_des.id;""", con = engine)
another_master

,id,track_name,size_bytes,price,user_rating,prime_genre,app_desc
0,281656475,PAC-MAN Premium,100788224,3.99,4.0,Games,"SAVE 20%, now only $3.99 for a limited time!\n..."
1,281656475,PAC-MAN Premium,100788224,3.99,4.0,Games,"SAVE 20%, now only $3.99 for a limited time!\n..."
2,281656475,PAC-MAN Premium,100788224,3.99,4.0,Games,"SAVE 20%, now only $3.99 for a limited time!\n..."
3,281796108,Evernote - stay organized,158578688,0.00,4.0,Productivity,Let Evernote change the way you organize your ...
4,281796108,Evernote - stay organized,158578688,0.00,4.0,Productivity,Let Evernote change the way you organize your ...
...,...,...,...,...,...,...,...
64768,1187838770,VR Roller Coaster World - Virtual Reality,97235968,0.00,4.5,Games,VR Roller Coaster World is an app for Google C...
64769,1187838770,VR Roller Coaster World - Virtual Reality,97235968,0.00,4.5,Games,VR Roller Coaster World is an app for Google C...
64770,1188375727,Escape the Sweet Shop Series,90898432,0.00,5.0,Games,5 previous escape games plus 1 new game in one...
64771,1188375727,Escape the Sweet Shop Series,90898432,0.00,5.0,Games,5 previous escape games plus 1 new game in one...
